# Exploring FinSpace's API
This notebook explores the FinSpace APIs, it is assumed you are accessing outside the FinSpace environment, using temporary credentials.

To run this notebook, you will need to bring:
- API Credentials (you can get this from FinSpace: gear menu -> API Credentials)
  - User must be in a group with the API Credentials permission
- a dataset ID

# THIS NOTEBOOK IS TO BE RUN OUTSIDE of FINSPACE


In [ ]:
import datetime as dt
import pandas as pd 
import boto3
import time
import pprint

from botocore.config import Config

In [ ]:
#### ---------------------------------------------------
#### REPLACE ID WITH ONE FROM YOUR ENVIRONMENT
#### ---------------------------------------------------
dataset_id = ''

#### ---------------------------------------------------
#### Identify your region
#### ---------------------------------------------------
region_name  = 'us-east-1'

### ----------------------------------------------------------------
### Get Credentials from the "API Credentials" in FinSpace 
### ----------------------------------------------------------------
hab_access_key_id     = ''
hab_secret_access_key = ''
hab_session_token     = ''


# Python Helper Class
This class wraps the service API so its easier to consume those results.

In [ ]:
 %load ../Utilities/finspace.py

## Initialize the Python Helper Class


In [ ]:
hab_session = boto3.session.Session(
    region_name           = region_name,
    aws_access_key_id     = hab_access_key_id,
    aws_secret_access_key = hab_secret_access_key,
    aws_session_token     = hab_session_token
)

finspace = FinSpace(
    boto_session = hab_session,
    dev_overrides = {
        "region_name" : region_name 
    }
)

# Everything about a dataset

## Dataset: Details

This service call will return top level details about the dataset, such as its name, description and schema.

In [ ]:
# get the specific dataset's details
dataset_details = finspace.describe_dataset_details(dataset_id = dataset_id)

#pprint.pprint( dataset_details, indent = 2)
pd.DataFrame.from_dict( dataset_details, 'index' )

In [ ]:
print( f"""
name        : {dataset_details['name']}
description : {dataset_details['description']}
kind        : {dataset_details['kind']}
created     : {dataset_details['createTimestamp']}
modified    : {dataset_details['modifiedTimestamp']}
""")

In [ ]:
schema = dataset_details['schemaV1']

print(f"""
Key Columns: {schema['primaryKeyColumns']}
""")

In [ ]:
pd.DataFrame.from_dict( schema['columns'] )

## Dataset: Changesets

In [ ]:
# changesets of the dataset
changesets = finspace.list_changesets(dataset_id = dataset_id)
achangeset = changesets[0]

#pprint.pprint(changesets)
pd.DataFrame.from_dict( changesets )

### A Changeset

In [ ]:
# one of the changesets
pd.DataFrame.from_dict( achangeset, 'index' )

### Contents of A Changeset

In [ ]:
# get the s3 location of the changeset
changeset_uri = achangeset['changesetURI']
changeset_uri

objs = finspace.list_objects(changeset_uri)
pd.DataFrame.from_dict(objs)

## Dataset: Views

In [ ]:
# get views
views = finspace.list_views(dataset_id)
aview = views[0]

#pprint.pprint(views, indent=2)
pd.DataFrame.from_dict( views )

### A View

In [ ]:
pd.DataFrame.from_dict( aview, 'index' )

# Permission Groups

In [ ]:
# get all permission groups
perm_group_list = finspace.list_permission_groups(maxResults = 50)

#pprint.pprint(perm_group_list)
pd.DataFrame.from_dict( perm_group_list )

## A Permission Group

In [ ]:
# A Group's permissions
perm_group = perm_group_list[0]

pd.DataFrame.from_dict( perm_group )

In [ ]:
# A group's permissions
perms = perm_group['applicationPermissions']

pd.DataFrame.from_dict( perms )

# Attribute Sets

In [ ]:
att_sets = finspace.list_dataset_types()

#pprint.pprint(dataset_types) 
pd.DataFrame.from_dict( att_sets )

In [ ]:
att_set = finspace.describe_attribute_set( att_sets[0]['id'])

# definition
pd.DataFrame.from_dict(att_set['definition'], 'index')

In [ ]:
# fields in the attribute set
att_fields = att_set['definition']['fields']

pd.DataFrame.from_dict(att_fields)

In [ ]:
# a field's type
att_field = att_fields[0]

pd.DataFrame.from_dict(att_field['type'], 'index')

# Classifications

In [ ]:
cls_list = finspace.list_classifications()

#pprint.pprint( cls_list )
pd.DataFrame.from_dict( cls_list )

In [ ]:
import datetime
print( f"Last Run: {dt.datetime.now()}" )